# NERSC Cluster Deploy Tutorial: Tuning Hyperparameters of a Distributed PyTorch Model with PBT using Ray Train & Tune

📖 [Back to Table of Contents](../README.md)<br>
<!-- ⬅ [Previous notebook](./ex_01_pytorch_ray_hvd.ipynb) <br> -->
<!-- ➡ [Next notebook](./ex_02_tensorflow_ray_train_tune.ipynb) <br> -->

----


## Introduction

We are going to run an example Ray Train & Tune code. This example looks at tunning hyperparameters of a distrbuted PyTorch Model with PBT. This tutorial is following the code in this example: https://docs.ray.io/en/latest/train/examples/pytorch/tune_cifar_torch_pbt_example.html

> **Note**:
> To setup the environment for the notebook, execute on command line: `./setup.sh 2` then select the kernel `pytorch-1.13.1` in the notebook

This Ray cluster will be setup using the NERSC PyTorch module and deployed on Perlmutter.



# Starting Ray Cluster

## Superfacility API

To deploy the Ray cluster via the NERSC Superfacility API you require a valid API client. 

To create a valid client visit your profile page in [Iris](https://iris.nersc.gov/):

<img src="img/iris_profile_header.png" width="800" />

Then scroll down to the **Superfacility API Clients** section and click the "+ New Client" button which will produce this window:

<img src="img/new_sf_api_client.png" width="400" />

To submit and deploy a Ray cluster we require the highest security level (<span style="color:red">RED</span>). **[This client id is valid for 2 days]**

Once created then saved the `client_id` string and `private_key` dictionary (you can also save the private key in PEM format) ready for use with the `SuperfacilityAPI` library.

> **Note**:
> This step should only be repeated if your client has expired


For more information about the NERSC Superfacility API visit the [documenation](https://docs.nersc.gov/services/sfapi/).

In [1]:
from SuperfacilityAPI import SuperfacilityAPI, SuperfacilityAccessToken
from utility import load_secrets

# Replace with your client id string and private key dictionary
client_id, private_key = load_secrets()
# client_id = "<your client id string>"
# private_key = "<your private key dict>"

api_key = SuperfacilityAccessToken(
    client_id = client_id,
    private_key = private_key
)
sfp_api = SuperfacilityAPI(api_key)

## Creating Ray Cluster

To create a ray cluster on NERSC compute nodes, execute the `deploy_ray_cluster` function with your desired slurm sbatch options.

In [2]:
from nersc_cluster_deploy import deploy_ray_cluster
from utility import user_account

slurm_options = {
    'qos': 'debug',
    'account': user_account(),
    'nodes': '3',
    't': '00:30:00'
}
site = 'perlmutter'
module_load = 'pytorch/1.13.1'

job = deploy_ray_cluster(
    sfp_api,
    slurm_options,
    site,
    job_setup = [f'module load {module_load}']
)

Now the job has been submitted, check on the job status

In [8]:
import os
import pandas as pd
sqs_table = sfp_api.get_jobs(site=site, user=os.getlogin(), sacct=False)
sqs_df = pd.DataFrame(sqs_table['output'])
sqs_df

,account,tres_per_node,min_cpus,min_tmp_disk,end_time,features,group,over_subscribe,jobid,name,...,partition,nodelist(reason),start_time,state,uid,submit_time,licenses,core_spec,schednodes,work_dir
0,dasrepo_g,N/A,128,0,2023-03-21T10:54:12,gpu&a100&hbm40g,75235,NO,6304042,sbatch,...,gpu_ss11,"nid[001164,001361,001364]",2023-03-21T10:24:12,RUNNING,75235,2023-03-21T10:19:37,u2:1,N/A,(null),/global/u2/a/asnaylor


Check job log

In [9]:
!cat ~/slurm-{job['jobid']}.out

In case of issues, please refer to our known issues: https://docs.nersc.gov/current/
and open a help ticket if your issue is not listed: https://help.nersc.gov/
[slurm] - Starting Ray HEAD
2023-03-21 10:24:24,850	INFO usage_lib.py:435 -- Usage stats collection is disabled.
2023-03-21 10:24:24,850	INFO scripts.py:710 -- Local node IP: nid001164
2023-03-21 10:24:26,520	SUCC scripts.py:747 -- --------------------
2023-03-21 10:24:26,520	SUCC scripts.py:748 -- Ray runtime started.
2023-03-21 10:24:26,520	SUCC scripts.py:749 -- --------------------
2023-03-21 10:24:26,520	INFO scripts.py:751 -- Next steps
2023-03-21 10:24:26,520	INFO scripts.py:752 -- To connect to this Ray runtime from another node, run
2023-03-21 10:24:26,520	INFO scripts.py:755 --   ray start --address='nid001164:6379'
2023-03-21 10:24:26,520	INFO scripts.py:771 -- Alternatively, use the following Python code:
2023-03-21 10:24:26,520	INFO scripts.py:773 -- import ray
2023-03-21 10:24:26,520	INFO scripts.py:777 -- ray.ini

## Connect to Ray + Grafana dashboards

Port forward both dashboards and get the URL paths

In [10]:
from nersc_cluster_deploy import connect_ray_dashboard


ray_dashboard_url, grafana_dashboard_url = connect_ray_dashboard(
    sfp_api,
    job['jobid'],
    site
)

In [11]:
ray_dashboard_url

'https://jupyter.nersc.gov/user/asnaylor/perlmutter-shared-node-cpu/proxy/localhost:8265/#/new/overview'

In [12]:
grafana_dashboard_url

'https://jupyter.nersc.gov/user/asnaylor/perlmutter-shared-node-cpu/proxy/3000/d/rayDefaultDashboard'

## Connect to Ray Cluster

Get the Ray cluster head node ip address to connect to the cluster

In [13]:
from nersc_cluster_deploy import get_ray_cluster_address
import ray

cluster_address = get_ray_cluster_address(
    sfp_api,
    job['jobid'],
    site
)
ray.init(cluster_address)

Python version:,3.9.15
Ray version:,2.3.0
Dashboard:,http://127.0.0.1:8265


Check all nodes connected to cluster

In [14]:
from nersc_cluster_deploy import ray_cluster_summary

ray_cluster_summary()

Cluster Summary
---------------
Nodes: 3
CPU:   318
GPU:   12
RAM:   465.96 GB


## Setup PyTorch Model

In [16]:
import argparse
import os

import torch
import torch.nn as nn
import torchvision.transforms as transforms
from filelock import FileLock
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import CIFAR10
from torchvision.models import resnet18

import ray
import ray.train as train
from ray import tune
from ray.air import session
from ray.air.checkpoint import Checkpoint
from ray.air.config import FailureConfig, RunConfig, ScalingConfig
from ray.train.torch import TorchTrainer
from ray.tune.schedulers import PopulationBasedTraining
from ray.tune.tune_config import TuneConfig
from ray.tune.tuner import Tuner

In [17]:
def train_epoch(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) // session.get_world_size()
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def validate_epoch(dataloader, model, loss_fn):
    size = len(dataloader.dataset) // session.get_world_size()
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n "
        f"Accuracy: {(100 * correct):>0.1f}%, "
        f"Avg loss: {test_loss:>8f} \n"
    )
    return {"loss": test_loss}


def update_optimizer_config(optimizer, config):
    for param_group in optimizer.param_groups:
        for param, val in config.items():
            param_group[param] = val


def train_func(config):
    epochs = config.get("epochs", 3)

    model = resnet18()

    # Note that `prepare_model` needs to be called before setting optimizer.
    if not session.get_checkpoint():  # fresh start
        model = train.torch.prepare_model(model)

    # Create optimizer.
    optimizer_config = {
        "lr": config.get("lr"),
        "momentum": config.get("momentum"),
    }
    optimizer = torch.optim.SGD(model.parameters(), **optimizer_config)

    starting_epoch = 0
    if session.get_checkpoint():
        checkpoint_dict = session.get_checkpoint().to_dict()

        # Load in model
        model_state = checkpoint_dict["model"]
        model.load_state_dict(model_state)
        model = train.torch.prepare_model(model)

        # Load in optimizer
        optimizer_state = checkpoint_dict["optimizer_state_dict"]
        optimizer.load_state_dict(optimizer_state)

        # Optimizer configs (`lr`, `momentum`) are being mutated by PBT and passed in
        # through config, so we need to update the optimizer loaded from the checkpoint
        update_optimizer_config(optimizer, optimizer_config)

        # The current epoch increments the loaded epoch by 1
        checkpoint_epoch = checkpoint_dict["epoch"]
        starting_epoch = checkpoint_epoch + 1

    # Load in training and validation data.
    transform_train = transforms.Compose(
        [
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ]
    )  # meanstd transformation

    transform_test = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ]
    )

    data_dir = config.get("data_dir", os.path.expanduser("~/data"))
    os.makedirs(data_dir, exist_ok=True)
    with FileLock(os.path.join(data_dir, ".ray.lock")):
        train_dataset = CIFAR10(
            root=data_dir, train=True, download=True, transform=transform_train
        )
        validation_dataset = CIFAR10(
            root=data_dir, train=False, download=False, transform=transform_test
        )

    if config.get("test_mode"):
        train_dataset = Subset(train_dataset, list(range(64)))
        validation_dataset = Subset(validation_dataset, list(range(64)))

    worker_batch_size = config["batch_size"] // session.get_world_size()

    train_loader = DataLoader(train_dataset, batch_size=worker_batch_size)
    validation_loader = DataLoader(validation_dataset, batch_size=worker_batch_size)

    train_loader = train.torch.prepare_data_loader(train_loader)
    validation_loader = train.torch.prepare_data_loader(validation_loader)

    # Create loss.
    criterion = nn.CrossEntropyLoss()

    for epoch in range(starting_epoch, epochs):
        train_epoch(train_loader, model, criterion, optimizer)
        result = validate_epoch(validation_loader, model, criterion)
        checkpoint = Checkpoint.from_dict(
            {
                "epoch": epoch,
                "model": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
            }
        )

        session.report(result, checkpoint=checkpoint)


## Train Model

In [18]:
SCRATCH = os.getenv('SCRATCH')

node_resources = ray.cluster_resources()
num_workers = int(node_resources['GPU'])
use_gpu = True

data_dir = os.path.join(SCRATCH, 'CIFAR10')
num_epochs = 5
smoke_test = False
synch = False

In [19]:
trainer = TorchTrainer(
        train_func,
        scaling_config=ScalingConfig(
            num_workers=num_workers, use_gpu=use_gpu
        ),
    )

In [20]:
pbt_scheduler = PopulationBasedTraining(
        time_attr="training_iteration",
        perturbation_interval=1,
        hyperparam_mutations={
            "train_loop_config": {
                # distribution for resampling
                "lr": tune.loguniform(0.001, 0.1),
                # allow perturbations within this set of categorical values
                "momentum": [0.8, 0.9, 0.99],
            }
        },
        synch=synch,
    )

In [21]:
tuner = Tuner(
        trainer,
        param_space={
            "train_loop_config": {
                "lr": tune.grid_search([0.001, 0.01, 0.05, 0.1]),
                "momentum": 0.8,
                "batch_size": 128 * num_workers,
                "test_mode": smoke_test,  # whether to to subset the data
                "data_dir": data_dir,
                "epochs": num_epochs,
            }
        },
        tune_config=TuneConfig(
            num_samples=1, metric="loss", mode="min", scheduler=pbt_scheduler
        ),
        run_config=RunConfig(
            stop={"training_iteration": 3 if smoke_test else num_epochs},
            failure_config=FailureConfig(max_failures=3),  # used for fault tolerance
        ),
    )


In [22]:
results = tuner.fit()

(TunerInternal pid=125486) /global/homes/a/asnaylor/.local/perlmutter/pytorch1.13.1/lib/python3.9/site-packages/ray/tune/tune.py:562: UserWarning: Consider boosting PBT performance by enabling `reuse_actors` as well as implementing `reset_config` for Trainable.
(TunerInternal pid=125486)   warnings.warn(
(TunerInternal pid=125486) 2023-03-21 10:26:35,092	WARNING trial_runner.py:369 -- The maximum number of pending trials has been automatically set to the number of available cluster CPUs, which is high (349 CPUs/pending trials). If you're running an experiment with a large number of trials, this could lead to scheduling overhead. In this case, consider setting the `TUNE_MAX_PENDING_TRIALS_PG` environment variable to the desired maximum number of concurrent trials.
(TunerInternal pid=125486) 2023-03-21 10:26:35,095	WARNING trial_runner.py:1677 -- You are trying to access _search_alg interface of TrialRunner in TrialScheduler, which is being restricted. If you believe it is reasonable for

(RayTrainWorker pid=126806, ip=128.55.66.67) Files already downloaded and verified
(RayTrainWorker pid=126807, ip=128.55.66.67) Files already downloaded and verified
(RayTrainWorker pid=126808, ip=128.55.66.67) Files already downloaded and verified
(RayTrainWorker pid=126809, ip=128.55.66.67) Files already downloaded and verified
(RayTrainWorker pid=93576, ip=128.55.67.168) Files already downloaded and verified
(RayTrainWorker pid=93579, ip=128.55.67.168) Files already downloaded and verified
(RayTrainWorker pid=93578, ip=128.55.67.168) Files already downloaded and verified
(RayTrainWorker pid=93577, ip=128.55.67.168) Files already downloaded and verified
(RayTrainWorker pid=125989) Files already downloaded and verified
(RayTrainWorker pid=125990) Files already downloaded and verified
(RayTrainWorker pid=125992) Files already downloaded and verified
(RayTrainWorker pid=125991) Files already downloaded and verified
(RayTrainWorker pid=93578, ip=128.55.67.168) loss: 6.940266  [    0/ 416

(RayTrainWorker pid=127168) 2023-03-21 10:27:17,174	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=12]
(RayTrainWorker pid=127821, ip=128.55.66.67) 2023-03-21 10:27:18,989	INFO train_loop_utils.py:255 -- Moving model to device: cuda:2
(RayTrainWorker pid=94097, ip=128.55.67.168) 2023-03-21 10:27:19,019	INFO train_loop_utils.py:255 -- Moving model to device: cuda:2
(RayTrainWorker pid=127168) 2023-03-21 10:27:19,005	INFO train_loop_utils.py:255 -- Moving model to device: cuda:2
(RayTrainWorker pid=127821, ip=128.55.66.67) 2023-03-21 10:27:20,501	INFO train_loop_utils.py:315 -- Wrapping provided model in DistributedDataParallel.
(RayTrainWorker pid=94097, ip=128.55.67.168) 2023-03-21 10:27:20,591	INFO train_loop_utils.py:315 -- Wrapping provided model in DistributedDataParallel.
(RayTrainWorker pid=127168) 2023-03-21 10:27:20,741	INFO train_loop_utils.py:315 -- Wrapping provided model in DistributedDataParallel.


(RayTrainWorker pid=127171) Files already downloaded and verified
(RayTrainWorker pid=127170) Files already downloaded and verified
(RayTrainWorker pid=127821, ip=128.55.66.67) Files already downloaded and verified
(RayTrainWorker pid=127823, ip=128.55.66.67) Files already downloaded and verified
(RayTrainWorker pid=127169) Files already downloaded and verified
(RayTrainWorker pid=127168) Files already downloaded and verified
(RayTrainWorker pid=94098, ip=128.55.67.168) Files already downloaded and verified
(RayTrainWorker pid=94097, ip=128.55.67.168) Files already downloaded and verified
(RayTrainWorker pid=127824, ip=128.55.66.67) Files already downloaded and verified
(RayTrainWorker pid=127822, ip=128.55.66.67) Files already downloaded and verified
(RayTrainWorker pid=94099, ip=128.55.67.168) Files already downloaded and verified
(RayTrainWorker pid=94100, ip=128.55.67.168) Files already downloaded and verified
(RayTrainWorker pid=127170) loss: 6.900143  [    0/ 4166]
(RayTrainWorke

(RayTrainWorker pid=128269) 2023-03-21 10:27:44,281	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=12]
(RayTrainWorker pid=128313, ip=128.55.66.67) 2023-03-21 10:27:46,099	INFO train_loop_utils.py:255 -- Moving model to device: cuda:2
(RayTrainWorker pid=128269) 2023-03-21 10:27:46,063	INFO train_loop_utils.py:255 -- Moving model to device: cuda:2
(RayTrainWorker pid=94589, ip=128.55.67.168) 2023-03-21 10:27:46,092	INFO train_loop_utils.py:255 -- Moving model to device: cuda:2
(RayTrainWorker pid=94589, ip=128.55.67.168) 2023-03-21 10:27:47,610	INFO train_loop_utils.py:315 -- Wrapping provided model in DistributedDataParallel.
(RayTrainWorker pid=128269) 2023-03-21 10:27:47,646	INFO train_loop_utils.py:315 -- Wrapping provided model in DistributedDataParallel.
(RayTrainWorker pid=128313, ip=128.55.66.67) 2023-03-21 10:27:47,721	INFO train_loop_utils.py:315 -- Wrapping provided model in DistributedDataParallel.


...

(RayTrainWorker pid=8237, ip=128.55.66.67) Files already downloaded and verified
(RayTrainWorker pid=8240, ip=128.55.66.67) Files already downloaded and verified
(RayTrainWorker pid=8238, ip=128.55.66.67) Files already downloaded and verified
(RayTrainWorker pid=8239, ip=128.55.66.67) Files already downloaded and verified
(RayTrainWorker pid=15086) Files already downloaded and verified
(RayTrainWorker pid=15088) Files already downloaded and verified
(RayTrainWorker pid=15084) Files already downloaded and verified
(RayTrainWorker pid=15089) Files already downloaded and verified
(RayTrainWorker pid=102852, ip=128.55.67.168) Files already downloaded and verified
(RayTrainWorker pid=102853, ip=128.55.67.168) Files already downloaded and verified
(RayTrainWorker pid=102851, ip=128.55.67.168) Files already downloaded and verified
(RayTrainWorker pid=102854, ip=128.55.67.168) Files already downloaded and verified
(RayTrainWorker pid=8240, ip=128.55.66.67) loss: 1.537392  [    0/ 4166]
(RayTra

(TunerInternal pid=125486) 2023-03-21 10:35:28,214	WARNING util.py:244 -- The `process_trial_save` operation took 0.975 s, which may be a performance bottleneck.
(TorchTrainer pid=16032) 2023-03-21 10:35:32,667	INFO trainable.py:791 -- Restored on nid001164 from checkpoint: /global/homes/a/asnaylor/ray_results/TorchTrainer_2023-03-21_10-26-33/TorchTrainer_876a2_00002_2_lr=0.0500_2023-03-21_10-27-38/checkpoint_tmpcc26c9
(TorchTrainer pid=16032) 2023-03-21 10:35:32,667	INFO trainable.py:800 -- Current state after restoring: {'_iteration': 4, '_timesteps_total': None, '_time_total': 93.77648591995239, '_episodes_total': None}
(RayTrainWorker pid=16180) 2023-03-21 10:35:35,764	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=12]
(RayTrainWorker pid=8949, ip=128.55.66.67) 2023-03-21 10:35:39,778	INFO train_loop_utils.py:255 -- Moving model to device: cuda:0
(RayTrainWorker pid=16180) 2023-03-21 10:35:39,808	INFO train_loop_utils.py:255 -- Moving model to device:

(RayTrainWorker pid=103369, ip=128.55.67.168) Files already downloaded and verified
(RayTrainWorker pid=103368, ip=128.55.67.168) Files already downloaded and verified
(RayTrainWorker pid=16183) Files already downloaded and verified
(RayTrainWorker pid=16180) Files already downloaded and verified
(RayTrainWorker pid=16182) Files already downloaded and verified
(RayTrainWorker pid=16181) Files already downloaded and verified
(RayTrainWorker pid=103370, ip=128.55.67.168) Files already downloaded and verified
(RayTrainWorker pid=103371, ip=128.55.67.168) Files already downloaded and verified
(RayTrainWorker pid=8951, ip=128.55.66.67) Files already downloaded and verified
(RayTrainWorker pid=8952, ip=128.55.66.67) Files already downloaded and verified
(RayTrainWorker pid=8949, ip=128.55.66.67) Files already downloaded and verified
(RayTrainWorker pid=8950, ip=128.55.66.67) Files already downloaded and verified
(RayTrainWorker pid=8952, ip=128.55.66.67) loss: 1.258440  [    0/ 4166]
(RayTra

(TorchTrainer pid=17557) 2023-03-21 10:35:59,933	INFO trainable.py:791 -- Restored on nid001164 from checkpoint: /global/homes/a/asnaylor/ray_results/TorchTrainer_2023-03-21_10-26-33/TorchTrainer_876a2_00003_3_lr=0.1000_2023-03-21_10-28-03/checkpoint_tmpe4edea
(TorchTrainer pid=17557) 2023-03-21 10:35:59,933	INFO trainable.py:800 -- Current state after restoring: {'_iteration': 4, '_timesteps_total': None, '_time_total': 93.14122271537781, '_episodes_total': None}
(RayTrainWorker pid=17707) 2023-03-21 10:36:02,920	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=12]
(RayTrainWorker pid=103865, ip=128.55.67.168) 2023-03-21 10:36:06,871	INFO train_loop_utils.py:255 -- Moving model to device: cuda:2
(RayTrainWorker pid=17707) 2023-03-21 10:36:06,893	INFO train_loop_utils.py:255 -- Moving model to device: cuda:2
(RayTrainWorker pid=9445, ip=128.55.66.67) 2023-03-21 10:36:06,894	INFO train_loop_utils.py:255 -- Moving model to device: cuda:2
(RayTrainWorker pid=1

(RayTrainWorker pid=103866, ip=128.55.67.168) Files already downloaded and verified
(RayTrainWorker pid=103868, ip=128.55.67.168) Files already downloaded and verified
(RayTrainWorker pid=9447, ip=128.55.66.67) Files already downloaded and verified
(RayTrainWorker pid=17708) Files already downloaded and verified
(RayTrainWorker pid=9445, ip=128.55.66.67) Files already downloaded and verified
(RayTrainWorker pid=17710) Files already downloaded and verified
(RayTrainWorker pid=17707) Files already downloaded and verified
(RayTrainWorker pid=9446, ip=128.55.66.67) Files already downloaded and verified
(RayTrainWorker pid=9448, ip=128.55.66.67) Files already downloaded and verified
(RayTrainWorker pid=103865, ip=128.55.67.168) Files already downloaded and verified
(RayTrainWorker pid=17709) Files already downloaded and verified
(RayTrainWorker pid=103867, ip=128.55.67.168) Files already downloaded and verified
(RayTrainWorker pid=9446, ip=128.55.66.67) loss: 1.201538  [    0/ 4166]
(RayTra

In [23]:
print(results.get_best_result(metric="loss", mode="min"))

Result(metrics={'loss': 1.1419129882540022, '_timestamp': 1679420184, '_time_this_iter_s': 19.26172709465027, '_training_iteration': 1, 'should_checkpoint': True, 'done': True, 'trial_id': '876a2_00003', 'experiment_tag': '3_lr=0.1000@perturbed[train_loop_config=lr_0_048_momentum_0_8_batch_size_1536_test_mode_False_data_dir_pscratch_sd_a_asnaylor_CIFAR10_epochs_5]'}, error=None, log_dir=PosixPath('/global/homes/a/asnaylor/ray_results/TorchTrainer_2023-03-21_10-26-33/TorchTrainer_876a2_00003_3_lr=0.1000_2023-03-21_10-28-03'))


## Close cluster conection and stop job

In [24]:
ray.shutdown()

(TunerInternal pid=125486) Result for TorchTrainer_876a2_00003:
(TunerInternal pid=125486)   _time_this_iter_s: 19.26172709465027
(TunerInternal pid=125486)   _timestamp: 1679420184
(TunerInternal pid=125486)   _training_iteration: 1
(TunerInternal pid=125486)   date: 2023-03-21_10-36-24
(TunerInternal pid=125486)   done: true
(TunerInternal pid=125486)   experiment_id: 2f564592309a4b05aee19a4ec9d1a599
(TunerInternal pid=125486)   hostname: nid001164
(TunerInternal pid=125486)   iterations_since_restore: 1
(TunerInternal pid=125486)   loss: 1.1419129882540022
(TunerInternal pid=125486)   node_ip: nid001164
(TunerInternal pid=125486)   pid: 17557
(TunerInternal pid=125486)   should_checkpoint: true
(TunerInternal pid=125486)   time_since_restore: 24.36621594429016
(TunerInternal pid=125486)   time_this_iter_s: 24.36621594429016
(TunerInternal pid=125486)   time_total_s: 117.50743865966797
(TunerInternal pid=125486)   timestamp: 1679420184
(TunerInternal pid=125486)   timesteps_since_res

(TunerInternal pid=125486) 2023-03-21 10:36:24,317	INFO tune.py:798 -- Total run time: 589.28 seconds (589.22 seconds for the tuning loop).


In [25]:
sfp_api.delete_job(site, job['jobid'])

Connection to nid001164 closed by remote host.
Connection to nid001164 closed by remote host.


{'task_id': '0', 'status': 'OK', 'error': None}

## Explore Training in Tensorboard

In [39]:
import nersc_tensorboard_helper
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [40]:
log_dir = str(results.get_best_result(metric="loss", mode="min").log_dir)

In [41]:
%tensorboard --logdir $log_dir --port 0

In [42]:
nersc_tensorboard_helper.tb_address()